In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [13]:
%sendtofile --source "../Sensor_Kennel/BME280_funcs.py"

Sent 77 lines (3294 bytes) to BME280_funcs.py.


In [2]:
from machine import I2C, Pin
import time, os, sys, ustruct

#i2c = I2C(1)  # on Pi
i2c = I2C(1, scl=Pin(25), sda=Pin(27)) # on ESP32 pin26 broken


print(list(map(hex, i2c.scan())))  # 0x23=LTR-559, 0x76=BME280, 0x49=ADS1015

['0x23', '0x49', '0x76']


In [3]:
import BME280_funcs
BME280_funcs.bme280init(i2c, 0x76)
for i in range(20):
    x = BME280_funcs.readBME280(0x76)
    print("temp, baro, humid", x)
    time.sleep_ms(500)

temp, baro, humid (20.04, 99018.86, 40.09571)
temp, baro, humid (20.03, 99019.86, 40.10644)
temp, baro, humid (20.04, 99015.39, 40.09571)
temp, baro, humid (20.03, 99011.08, 40.08301)
temp, baro, humid (20.04, 99020.68, 40.07226)
temp, baro, humid (20.03, 99016.38, 40.07129)
temp, baro, humid (20.04, 99018.03, 40.11817)
temp, baro, humid (20.03, 99011.91, 40.1416)
temp, baro, humid (20.04, 99018.03, 40.15332)
temp, baro, humid (20.04, 99018.86, 40.1543)
temp, baro, humid (20.04, 99018.03, 40.12989)
temp, baro, humid (20.03, 99014.56, 40.12989)
temp, baro, humid (20.02, 99018.2, 40.10644)
temp, baro, humid (20.03, 99017.2, 40.10644)
temp, baro, humid (20.02, 99023.49, 40.09473)
temp, baro, humid (20.03, 99026.99, 40.07129)
temp, baro, humid (20.03, 99033.11, 40.07226)
temp, baro, humid (20.03, 99027.81, 40.07226)
temp, baro, humid (20.03, 99026.99, 40.08301)
temp, baro, humid (20.02, 99031.45, 40.05957)


In [59]:
print("LTR-559 part_number/revision", i2c.readfrom_mem(0x23, 0x86, 1), "should be 9 2")
i2c.writeto_mem(0x23, 0x80, b'\x09')  # enable ALS gain x4 (light sensor)
for i in range(20):
    x = ustruct.unpack("h", i2c.readfrom_mem(0x23, 0x8A, 2))
    print(x)
    time.sleep_ms(500)

LTR-559 part_number/revision b'\x92' should be 9 2
(3112,)
(3106,)
(3100,)
(3123,)
(3489,)
(4414,)
(3875,)
(3855,)
(4156,)
(3956,)
(3914,)
(3947,)
(3876,)
(44,)
(48,)
(47,)
(47,)
(47,)
(47,)
(47,)


In [21]:
# Numbers all seem wrong coming out here (should attach 3.3V to the heater enable pin)

# start conversion | channel | gain(FSR) | single shot | 1600 SPS | disable comparator
channels = [0x4000, 0x5000, 0x6000, 0x7000]  # in0/gnd, in1, in2, in3 (fixed at 1.25V reference)
gainfsr, gainV = 0x0000, 6.144
for i in range(20):
    vals = [ ]
    for channel in channels:
        i2c.writeto_mem(0x49, 0x01, ustruct.pack(">H", 0x8000 | channel | gainfsr | 0x0100 | 0x0080 | 0x0003))
        time.sleep_ms(10)
        #for j in range(10000):
        #    x = i2c.readfrom_mem(0x49, 0x01, 2)
        #    x1 = ustruct.unpack(">H", x)[0]
        #    if (x1 & 0x8000):  # conversion complete
        #        break
        r = ustruct.unpack(">h", i2c.readfrom_mem(0x49, 0x00, 2))
        #vals.append(r[0]>>3)
        v = (r[0] >> 3)/2048.0*gainV
        vals.append(v)
    print("ADS1015", vals)
    #print("ADS1015", "ox", 56000*(3.3-vals[0]), "red", 56000*(3.3-vals[1]), "nh3", 56000*(3.3-vals[2]), "ref", vals[3])
    time.sleep_ms(500)

ADS1015 [2.64, 5.796, 4.2, 1.044]
ADS1015 [2.634, 5.802, 4.2, 1.068]
ADS1015 [2.634, 5.796, 4.2, 1.236]
ADS1015 [2.64, 5.802, 4.2, 1.086]
.ADS1015 [2.634, 5.802, 4.2, 1.026]
ADS1015 [2.64, 5.802, 4.2, 1.122]
ADS1015 [2.64, 5.802, 4.2, 1.224]
ADS1015 [2.64, 5.802, 4.2, 1.05]
ADS1015 [2.64, 5.802, 4.2, 1.044]
ADS1015 [2.64, 5.802, 4.2, 1.2]
ADS1015 [2.64, 5.802, 4.2, 1.146]
ADS1015 [2.64, 5.802, 4.2, 1.056]
ADS1015 [2.64, 5.802, 4.2, 1.062]
ADS1015 [2.64, 5.802, 4.2, 1.236]
.ADS1015 [2.64, 5.802, 4.2, 1.068]
ADS1015 [2.64, 5.802, 4.2, 1.026]
ADS1015 [2.64, 5.802, 4.2, 1.152]
ADS1015 [2.64, 5.802, 4.2, 1.188]
ADS1015 [2.64, 5.802, 4.2, 1.056]
ADS1015 [2.64, 5.802, 4.2, 1.044]
